In [1]:
from torch.ao.nn.quantized.functional import threshold

from models.microscopy_cnn import MicroscopyCNN
from scripts.model_metrics import score_model
import torch
from scripts.dataset_loader import MicroscopyDataset
from torch.utils.data import DataLoader

MicroscopyCNN(
  (conv1): Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=262144, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
MicroscopyCNN(
  (conv1): Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(

In [ ]:
model = MicroscopyCNN(in_channels=2, task='classification')

# model.load_state_dict(torch.load(r'C:\Users\nmp002\PycharmProjects\MPM-MachineLearning\trained_models\best_NADH_only_24FEB2025.pt', map_location=torch.device('cpu'),weights_only=True))
model.load_state_dict(torch.load(r'C:\Users\nmp002\PycharmProjects\MPM-MachineLearning\trained_models\epoch250_NADH_SHG.pt', map_location=torch.device('cpu'),weights_only=True))

In [ ]:
# Load data
csv_file=r"C:\Users\nmp002\PycharmProjects\MPM-MachineLearning\data\newData\labels.csv"
root_dir=r"C:\Users\nmp002\PycharmProjects\MPM-MachineLearning\data\newData"

dataset = MicroscopyDataset(
    csv_file=csv_file,
    root_dir=root_dir,
    # channels = ['nadh'],
    channels = ['nadh', 'shg'],
    transform=None
)

In [ ]:
specific_test_paths = []
# sample_indices = [23,0,3,20]
sample_indices = range(0,29)
eval_indices = [27, 4, 25, 7, 8]
for sample_index in eval_indices:
    if sample_index not in [25,7]:
        specific_test_paths.append(dataset.sample_wise_paths[sample_index])

print(specific_test_paths)
# print('\n------------------------------------------\n')
# print(dataset.sample_wise_paths)

In [ ]:
from sklearn.metrics import roc_auc_score


def score_em(t, o):
    fpr, tpr, thresholds = roc_curve(t, o)
    score = auc(fpr, tpr)
    score_all = roc_auc_score(t, o)
    print(score_all)
    thresh = thresholds[np.argmax(tpr - fpr)]
    print(type(thresholds), type(thresh))
    preds = [out >= thresh for out in o]
    for tar,pred in zip(t, preds):
        print(f'{tar} -> {pred}')
    RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=score).plot()
    ConfusionMatrixDisplay.from_predictions(t, preds).plot()

In [ ]:
from sklearn.metrics import auc, roc_curve, confusion_matrix, RocCurveDisplay, ConfusionMatrixDisplay
from scripts.dataset_loader import tiff_to_tensor
import numpy as np
with torch.no_grad():
    model.eval()
    targets = []
    all_targets = []
    outs = []
    all_outs = []
    for sample in specific_test_paths:
        fov_outs = []
        if sample:
            for fov in sample:
                # De-nest fov paths and get the indexed item path
                score = fov[1]
                combined_image = torch.cat([tiff_to_tensor(channel) for channel in fov[0]], dim=0).unsqueeze(0)
                y = model(combined_image).item()
                all_outs.append(y)
                fov_outs.append(y)
                all_targets.append(1 if score > 25 else 0)
            targets.append(1 if score > 25 else 0)
            outs.append(np.mean(fov_outs))
score_em(all_targets, all_outs)
score_em(targets, outs)

In [ ]:
print(all_outs)

In [ ]:
# best NADH_SHG = 0.5153061224489796
# epoch500 NADH_SHG =

In [ ]:
print(fov_outs)

In [ ]:
print(targets)

In [ ]:
print(outs)

In [17]:
input_channels = ['nadh','shg']
# Load full dataset
full_dataset = MicroscopyDataset(
    csv_file="../data/newData/labels.csv",
    root_dir="../data/newData",
    channels = input_channels,
    transform=None
)

Total samples found: 290


In [18]:
print(full_dataset.sample_wise_paths)

[[(['../data/newData\\Sample_001\\fov1\\nadh.tiff', '../data/newData\\Sample_001\\fov1\\shg.tiff'], 77), (['../data/newData\\Sample_001\\fov2\\nadh.tiff', '../data/newData\\Sample_001\\fov2\\shg.tiff'], 77), (['../data/newData\\Sample_001\\fov3\\nadh.tiff', '../data/newData\\Sample_001\\fov3\\shg.tiff'], 77), (['../data/newData\\Sample_001\\fov4\\nadh.tiff', '../data/newData\\Sample_001\\fov4\\shg.tiff'], 77), (['../data/newData\\Sample_001\\fov5\\nadh.tiff', '../data/newData\\Sample_001\\fov5\\shg.tiff'], 77)], [(['../data/newData\\Sample_002\\fov1\\nadh.tiff', '../data/newData\\Sample_002\\fov1\\shg.tiff'], 28), (['../data/newData\\Sample_002\\fov2\\nadh.tiff', '../data/newData\\Sample_002\\fov2\\shg.tiff'], 28), (['../data/newData\\Sample_002\\fov3\\nadh.tiff', '../data/newData\\Sample_002\\fov3\\shg.tiff'], 28), (['../data/newData\\Sample_002\\fov4\\nadh.tiff', '../data/newData\\Sample_002\\fov4\\shg.tiff'], 28), (['../data/newData\\Sample_002\\fov5\\nadh.tiff', '../data/newData\\S

In [3]:
samples_list = full_dataset.sample_wise_paths
print(samples_list)

[[(['../data/newData\\Sample_001\\fov1\\nadh.tiff', '../data/newData\\Sample_001\\fov1\\shg.tiff'], 77), (['../data/newData\\Sample_001\\fov2\\nadh.tiff', '../data/newData\\Sample_001\\fov2\\shg.tiff'], 77), (['../data/newData\\Sample_001\\fov3\\nadh.tiff', '../data/newData\\Sample_001\\fov3\\shg.tiff'], 77), (['../data/newData\\Sample_001\\fov4\\nadh.tiff', '../data/newData\\Sample_001\\fov4\\shg.tiff'], 77), (['../data/newData\\Sample_001\\fov5\\nadh.tiff', '../data/newData\\Sample_001\\fov5\\shg.tiff'], 77)], [(['../data/newData\\Sample_002\\fov1\\nadh.tiff', '../data/newData\\Sample_002\\fov1\\shg.tiff'], 28), (['../data/newData\\Sample_002\\fov2\\nadh.tiff', '../data/newData\\Sample_002\\fov2\\shg.tiff'], 28), (['../data/newData\\Sample_002\\fov3\\nadh.tiff', '../data/newData\\Sample_002\\fov3\\shg.tiff'], 28), (['../data/newData\\Sample_002\\fov4\\nadh.tiff', '../data/newData\\Sample_002\\fov4\\shg.tiff'], 28), (['../data/newData\\Sample_002\\fov5\\nadh.tiff', '../data/newData\\S

In [4]:
import random
random.shuffle(samples_list)
print(samples_list)

[[(['../data/newData\\Sample_008\\fov1\\nadh.tiff', '../data/newData\\Sample_008\\fov1\\shg.tiff'], 7), (['../data/newData\\Sample_008\\fov2\\nadh.tiff', '../data/newData\\Sample_008\\fov2\\shg.tiff'], 7), (['../data/newData\\Sample_008\\fov3\\nadh.tiff', '../data/newData\\Sample_008\\fov3\\shg.tiff'], 7), (['../data/newData\\Sample_008\\fov4\\nadh.tiff', '../data/newData\\Sample_008\\fov4\\shg.tiff'], 7), (['../data/newData\\Sample_008\\fov5\\nadh.tiff', '../data/newData\\Sample_008\\fov5\\shg.tiff'], 7)], [(['../data/newData\\Sample_012\\fov1\\nadh.tiff', '../data/newData\\Sample_012\\fov1\\shg.tiff'], 10), (['../data/newData\\Sample_012\\fov2\\nadh.tiff', '../data/newData\\Sample_012\\fov2\\shg.tiff'], 10), (['../data/newData\\Sample_012\\fov3\\nadh.tiff', '../data/newData\\Sample_012\\fov3\\shg.tiff'], 10), (['../data/newData\\Sample_012\\fov4\\nadh.tiff', '../data/newData\\Sample_012\\fov4\\shg.tiff'], 10), (['../data/newData\\Sample_012\\fov5\\nadh.tiff', '../data/newData\\Sample

In [5]:
import torchvision.transforms.v2 as tvt
# Transformations for training set
train_transform = tvt.Compose([
    tvt.RandomVerticalFlip(p=0.25),
    tvt.RandomHorizontalFlip(p=0.25),
    tvt.RandomRotation(degrees=(-180, 180))])
full_dataset.transform = train_transform

In [6]:
total_samples = len(samples_list)
train_size = int(0.7 * total_samples)
val_size = int(0.2 * total_samples)
test_size = total_samples - train_size - val_size

indices = torch.utils.data.SubsetRandomSampler(range(len(samples_list)))
indices = [i for i in indices]
for i in indices:
    print(f'Sample {i} is class -> {samples_list[i][0][1] > 25}')

Sample 9 is class -> True
Sample 7 is class -> True
Sample 10 is class -> True
Sample 11 is class -> True
Sample 25 is class -> False
Sample 20 is class -> False
Sample 8 is class -> True
Sample 13 is class -> True
Sample 16 is class -> False
Sample 17 is class -> False
Sample 2 is class -> False
Sample 22 is class -> False
Sample 28 is class -> True
Sample 18 is class -> False
Sample 14 is class -> False
Sample 0 is class -> False
Sample 3 is class -> False
Sample 21 is class -> True
Sample 19 is class -> True
Sample 24 is class -> True
Sample 23 is class -> False
Sample 1 is class -> False
Sample 6 is class -> False
Sample 27 is class -> False
Sample 4 is class -> True
Sample 5 is class -> True
Sample 12 is class -> True
Sample 15 is class -> True
Sample 26 is class -> False


In [19]:
sample_id = full_dataset.data_frame['sample_id'].iloc[9]
for i, fov_path in enumerate(full_dataset._denest()):
        print(fov_path[0][0])

../data/newData\Sample_001\fov1\nadh.tiff
../data/newData\Sample_001\fov2\nadh.tiff
../data/newData\Sample_001\fov3\nadh.tiff
../data/newData\Sample_001\fov4\nadh.tiff
../data/newData\Sample_001\fov5\nadh.tiff
../data/newData\Sample_002\fov1\nadh.tiff
../data/newData\Sample_002\fov2\nadh.tiff
../data/newData\Sample_002\fov3\nadh.tiff
../data/newData\Sample_002\fov4\nadh.tiff
../data/newData\Sample_002\fov5\nadh.tiff
../data/newData\Sample_003\fov1\nadh.tiff
../data/newData\Sample_003\fov2\nadh.tiff
../data/newData\Sample_003\fov3\nadh.tiff
../data/newData\Sample_003\fov4\nadh.tiff
../data/newData\Sample_003\fov5\nadh.tiff
../data/newData\Sample_004\fov1\nadh.tiff
../data/newData\Sample_004\fov2\nadh.tiff
../data/newData\Sample_004\fov3\nadh.tiff
../data/newData\Sample_004\fov4\nadh.tiff
../data/newData\Sample_004\fov5\nadh.tiff
../data/newData\Sample_005\fov1\nadh.tiff
../data/newData\Sample_005\fov2\nadh.tiff
../data/newData\Sample_005\fov3\nadh.tiff
../data/newData\Sample_005\fov4\na

In [49]:
print(indices)
train_samples = indices[:train_size]
print(f"Train samples: {train_samples}")
train_indices = [full_dataset.get_sample_indices(sample) for sample in train_samples]
train_indices = [i for sublist in train_indices for i in sublist]
print(f"Train indices: {train_indices}")
train_data = torch.utils.data.Subset(full_dataset.sample_wise_paths, train_indices)


val_samples = indices[train_size:train_size+val_size]
print(f"Val samples: {val_samples}")
val_indices = [full_dataset.get_sample_indices(sample) for sample in val_samples]
val_indices = [i for sublist in val_indices for i in sublist]
print(f"Val indices: {val_indices}")
val_data = torch.utils.data.Subset(full_dataset.sample_wise_paths, val_indices)

test_samples = indices[train_size+val_size:]
print(f"Test samples: {test_samples}")
test_indices = [full_dataset.get_sample_indices(sample) for sample in test_samples]
test_indices = [i for sublist in test_indices for i in sublist]
print(f"Test indices: {test_indices}")
test_data = torch.utils.data.Subset(full_dataset.sample_wise_paths, test_indices)

[9, 7, 10, 11, 25, 20, 8, 13, 16, 17, 2, 22, 28, 18, 14, 0, 3, 21, 19, 24, 23, 1, 6, 27, 4, 5, 12, 15, 26]
Train samples: [9, 7, 10, 11, 25, 20, 8, 13, 16, 17, 2, 22, 28, 18, 14, 0, 3, 21, 19, 24]
Train indices: [45, 46, 47, 48, 49, 35, 36, 37, 38, 39, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 125, 126, 127, 128, 129, 100, 101, 102, 103, 104, 40, 41, 42, 43, 44, 65, 66, 67, 68, 69, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 10, 11, 12, 13, 14, 110, 111, 112, 113, 114, 140, 141, 142, 143, 144, 90, 91, 92, 93, 94, 70, 71, 72, 73, 74, 0, 1, 2, 3, 4, 15, 16, 17, 18, 19, 105, 106, 107, 108, 109, 95, 96, 97, 98, 99, 120, 121, 122, 123, 124]
Val samples: [23, 1, 6, 27, 4]
Val indices: [115, 116, 117, 118, 119, 5, 6, 7, 8, 9, 30, 31, 32, 33, 34, 135, 136, 137, 138, 139, 20, 21, 22, 23, 24]
Test samples: [5, 12, 15, 26]
Test indices: [25, 26, 27, 28, 29, 60, 61, 62, 63, 64, 75, 76, 77, 78, 79, 130, 131, 132, 133, 134]


In [50]:
dataloaders = {
    'train': DataLoader(train_data, batch_size=16, shuffle=True),
    'val': DataLoader(val_data, batch_size=16, shuffle=False),
    'test': DataLoader(test_data, batch_size=16, shuffle=False)
}

In [54]:
for id_num in train_samples:
    print(f"Sample_{(id_num+1):03}")

Sample_010
Sample_008
Sample_011
Sample_012
Sample_026
Sample_021
Sample_009
Sample_014
Sample_017
Sample_018
Sample_003
Sample_023
Sample_029
Sample_019
Sample_015
Sample_001
Sample_004
Sample_022
Sample_020
Sample_025


In [ ]:
total_samples = len(samples_list)
train_size = int(0.7 * total_samples)
val_size = int(0.2 * total_samples)
test_size = total_samples - train_size - val_size
print(f"Total samples: {total_samples}, train_size: {train_size}, val_size: {val_size}, test_size: {test_size}")

In [ ]:
train_samples = samples_list[:train_size]
val_samples = samples_list[train_size:train_size+val_size]
test_samples = samples_list[train_size+val_size:]
print(f"Train samples: {train_samples}\n\nval samples: {val_samples}\n\ntest samples: {test_samples}")

In [61]:
for sample in test_data:
    if sample:
        for fov in sample:
            score = fov[1]
            print(score)

30
30
30
30
30
28
28
28
28
28
37
37
37
37
37
8
8
8
8
8


In [ ]:
p